# DrawingBoard

In [1]:
!export CUDA_VISIBLE_DEVICES=1

In [2]:
import tensorflow as tf
from inception import image_processing
from inception import inception_model as inception
from inception.skin_data import SkinData

import os

In [3]:
# load pb and analyze
output_graph_path = '/home/esteva/tensorflow_inception_graph.pb'

with tf.Graph().as_default():
    output_graph_def = tf.GraphDef()
    with open(output_graph_path, "rb") as f:
        output_graph_def.ParseFromString(f.read())
        _ = tf.import_graph_def(output_graph_def, name="")

In [12]:
# So the inception pb has 'input' and 'output' node names, which match the iOS code's extracted nodes.
names = []
for i, node in enumerate(output_graph_def.node):
    names.append(node.name)
names
    
    

[u'input',
 u'conv2d0_w',
 u'conv2d0_b',
 u'conv2d1_w',
 u'conv2d1_b',
 u'conv2d2_w',
 u'conv2d2_b',
 u'mixed3a_1x1_w',
 u'mixed3a_1x1_b',
 u'mixed3a_3x3_bottleneck_w',
 u'mixed3a_3x3_bottleneck_b',
 u'mixed3a_3x3_w',
 u'mixed3a_3x3_b',
 u'mixed3a_5x5_bottleneck_w',
 u'mixed3a_5x5_bottleneck_b',
 u'mixed3a_5x5_w',
 u'mixed3a_5x5_b',
 u'mixed3a_pool_reduce_w',
 u'mixed3a_pool_reduce_b',
 u'mixed3b_1x1_w',
 u'mixed3b_1x1_b',
 u'mixed3b_3x3_bottleneck_w',
 u'mixed3b_3x3_bottleneck_b',
 u'mixed3b_3x3_w',
 u'mixed3b_3x3_b',
 u'mixed3b_5x5_bottleneck_w',
 u'mixed3b_5x5_bottleneck_b',
 u'mixed3b_5x5_w',
 u'mixed3b_5x5_b',
 u'mixed3b_pool_reduce_w',
 u'mixed3b_pool_reduce_b',
 u'mixed4a_1x1_w',
 u'mixed4a_1x1_b',
 u'mixed4a_3x3_bottleneck_w',
 u'mixed4a_3x3_bottleneck_b',
 u'mixed4a_3x3_w',
 u'mixed4a_3x3_b',
 u'mixed4a_5x5_bottleneck_w',
 u'mixed4a_5x5_bottleneck_b',
 u'mixed4a_5x5_w',
 u'mixed4a_5x5_b',
 u'mixed4a_pool_reduce_w',
 u'mixed4a_pool_reduce_b',
 u'mixed4b_1x1_w',
 u'mixed4b_1x1_b

In [5]:
FLAGS = tf.app.flags.FLAGS
FLAGS.checkpoint_dir = '/media/esteva/ExtraDrive1/ThrunResearch/tf_experiments/nine-way/train'
FLAGS.data_dir = '/media/esteva/ExtraDrive1/ThrunResearch/data/skindata4/tf_shards/nine-way'

dataset = SkinData(subset='validation', num_classes=9)

In [6]:
def evaluate(dataset):
    """Evaluate model on Dataset for a number of steps."""
    with tf.Graph().as_default() as g:
        # Get images and labels from the dataset.
        images, labels = image_processing.inputs(dataset)

        # Number of classes in the Dataset label set plus 1.
        # Label 0 is reserved for an (unused) background class.
        num_classes = dataset.num_classes() + 1

        # Build a Graph that computes the logits predictions from the
        # inference model.
        logits, _ = inception.inference(images, num_classes)
        softmax = tf.nn.softmax(logits)   

        # Restore the moving average version of the learned variables for eval.
        variable_averages = tf.train.ExponentialMovingAverage(
            inception.MOVING_AVERAGE_DECAY)
        variables_to_restore = variable_averages.variables_to_restore()
        saver = tf.train.Saver(variables_to_restore)

        with tf.Session() as sess:
            ckpt = tf.train.get_checkpoint_state(FLAGS.checkpoint_dir)
            if ckpt and ckpt.model_checkpoint_path:
                if os.path.isabs(ckpt.model_checkpoint_path):
                    # Restores from checkpoint with absolute path.
                    saver.restore(sess, ckpt.model_checkpoint_path)
                else:
                    # Restores from checkpoint with relative path.
                    saver.restore(sess, os.path.join(FLAGS.checkpoint_dir,
                                                     ckpt.model_checkpoint_path))

                # Assuming model_checkpoint_path looks something like:
                #   /my-favorite-path/imagenet_train/model.ckpt-0,
                # extract global_step from it.
                global_step = ckpt.model_checkpoint_path.split('/')[-1].split('-')[-1]
                print('Succesfully loaded model from %s at step=%s.' %
                    (ckpt.model_checkpoint_path, global_step))
            else:
                print('No checkpoint file found')
        return g
    

g = evaluate(dataset)

Succesfully loaded model from model.ckpt-80000 at step=80000.


In [9]:
gd = g.as_graph_def()

names = []
for node in gd.node:
    names.append(node.name)


In [13]:
names[-30:]

[u'nn0_pre_relu/matmul',
 u'nn0_pre_relu',
 u'nn0',
 u'nn0/reshape/shape',
 u'nn0/reshape',
 u'softmax0_pre_activation/matmul',
 u'softmax0_pre_activation',
 u'softmax0',
 u'head1_pool',
 u'head1_bottleneck_pre_relu/conv',
 u'head1_bottleneck_pre_relu',
 u'head1_bottleneck',
 u'head1_bottleneck/reshape/shape',
 u'head1_bottleneck/reshape',
 u'nn1_pre_relu/matmul',
 u'nn1_pre_relu',
 u'nn1',
 u'nn1/reshape/shape',
 u'nn1/reshape',
 u'softmax1_pre_activation/matmul',
 u'softmax1_pre_activation',
 u'softmax1',
 u'avgpool0/reshape/shape',
 u'avgpool0/reshape',
 u'softmax2_pre_activation/matmul',
 u'softmax2_pre_activation',
 u'softmax2',
 u'output',
 u'output1',
 u'output2']

In [11]:
names

[u'batch_processing/input_producer/Const',
 u'batch_processing/input_producer/Size',
 u'batch_processing/input_producer/Greater/y',
 u'batch_processing/input_producer/Greater',
 u'batch_processing/input_producer/Assert/data_0',
 u'batch_processing/input_producer/Assert',
 u'batch_processing/input_producer/Identity',
 u'batch_processing/input_producer',
 u'batch_processing/input_producer/input_producer_EnqueueMany',
 u'batch_processing/input_producer/input_producer_Close',
 u'batch_processing/input_producer/input_producer_Close_1',
 u'batch_processing/input_producer/input_producer_Size',
 u'batch_processing/input_producer/Cast',
 u'batch_processing/input_producer/mul/y',
 u'batch_processing/input_producer/mul',
 u'batch_processing/input_producer/ScalarSummary/tags',
 u'batch_processing/input_producer/ScalarSummary',
 u'batch_processing/fifo_queue',
 u'batch_processing/TFRecordReader',
 u'batch_processing/ReaderRead',
 u'batch_processing/ParseSingleExample/ExpandDims/dim',
 u'batch_proce